# Tutorial to Create NNR
This project requires that you source the setup_env.sh script before executing anything below.

## Creating the Giant Spreadsheets

First you need to make the dataset of AERONET-MODIS co-locations. <br>
Run the run_create_giant.sh script. This calls the python code create_giant.py which will create two files in /nobackup/NNR/Training/061 - one each for Aqua and Terra:
- giant_C061_10km_Aqua_v3.0_20211231.nc  
- giant_C061_10km_Terra_v3.0_20211231.nc

The script will run through the entire MODIS data record up until the end date indicated, in this case 2021-12-31. <br>
You can modify the end date by editing the -E option inside the run_create_giant.sh script.

This script will take several days to run.

The script defaults are to use MODIS collection 061 and AERONET version 3.

The script co-locates the data according to the following criteria:
- MODIS observations are within 27.5 km of an AERONET site
- AERONET observations are within 30 minutes of MODIS overpass

### Description of what create_giant.py does:
creat_giant.py relies on two pyobs libraries: mxd04.py and aeronet.py to read the MODIS and AERONET files

Important caveat of aeronet.py is that the AOD values at 550, 660, 670, and 470 nm are angstrom interpolated from measurements at neighboring wavelengths. When these values are returned to create_giant.py, it does a check to see that AOD at 550 nm is greater than zero. It then filters out any observations that don't meet that criteria.

The mxd04.py script reads in the MODIS granules, and is run with the only_good option set to True.
This filters the observations according to the MODIS reported QA Flag. The only_good threshholds are different for each retrieval.
- BAD, MARGINAL, GOOD, BEST = ( 0, 1, 2, 3 )
- LAND: qa_flag == BEST 
- OCEAN: qa_flag  > BAD
- DEEP: qa_flag > BAD

After this data is passed back to create_giant.py, additional filters are applied.
1. Cloud fraction < 0.7
2. All reported TOA reflectance values > 0
3. All reported surface reflectance values > 0
4. OCEAN: glint angle > 40 <br>
   LAND & DEEP: scattering angle < 170

create_giant.py then does a co-location between the MODIS obs and the AERONET obs according to the criteria above.
It calculates the mean, mode, and standard deviation of the respective co-located observations. 

There need to be atleast 2 AERONET obs and 5 MODIS obs to make a match.

The create_giant.py script uses the header information from the text file giantHeader.txt to get the variable names and attributes to write to a netCDF file.  The giantHeader.txt file was generated from one of the giant spreadsheet files created by the MODIS team and so closely follows that format.


## Sampling Other Datasets

Once the giant spreadsheet file is created, you need to sample other datasets at the MODIS-AERONET colocations.

### To sample MERRA-2
Run the run_sample_merra.sh script. This script calls the python code sample_merra.py that requires the name of the giant spreadsheet file as input. The names of the giant spreadsheet files are hardcoded in run_sample_merra.sh.

sample_merra.py relies on the GIANT class to read the giant spreadsheet to get the lat,lon, and times of the co-locations, and calls the function 'sampleMERRA' attached to the GIANT class. 
sample_merra.py defines the name of the output .npz file from the giant spreadsheet filename.
sample_merra.py assumes that you have two control files located in your directory:
1. tavg1_2d_aer_Nx
2. tavg1_2d_slv_Nx

the sampleMERRA function will read the following variables from these two file collections
- tavg1_2d_aer_Nx: 'TOTEXTTAU','DUEXTTAU','SSEXTTAU','BCEXTTAU','OCEXTTAU','SUEXTTAU'
- tavg1_2d_slv_Nx: V10M, U10M

It then calculates 10M wind speed, and the fraction AOD of the species. Note BC and OC are summed together to get fraction carbonaceous aerosol (acronym 'CC').

The windspeed, V10M, U10M, and AOD fractions are written to the aforementioned npz file. 

### To sample MCD43C1
First ensure that the MCD43C1 dataset is downloaded.  Run the run_download_mcd43c1.sh script. This script calls mcd43c_download.py.  This python code has an option called --root which sets the variable rootDir, this is the location of the dataset. Default is /nobackup/3/pcastell/MODIS/MCD43C1/061.
The program loops through a date range given and will look in rootDir. If it doesn't find a file for a day it will download it from https://e4ftl01.cr.usgs.gov/MOTA/MCD43C1.061/

Next, you have to sample the MCD43C1 data by running the run_sample_mcd43c1.sh script. This script calls the code sample_mcd43c1.py that requires the name of the giant spreadsheet as input. The names of the giant spreadsheet files are hardcoded in run_sample_mcd43c1.sh.

sample_mcd43c1.py relies on the GIANT class to read the giant spreadsheet to get the lat, lon, and times of the co-location, and calls the function 'sampleMCD43C' attached to the giant class. Note that the way sample_mcd43c1.py is currently set up it implicitly assumes that you're using the detault rootDir above, but you could pass the sampleMCD43C function the optional variable "inDir" if the data is in another directory.  After sampling, the function writes the values to an npz file defined in sample_mcd43c1.py that's based on the giant spreadsheet filename.

### To calculate Cox-Munk BRDF
First ensure that the MERRA-2 wind speeds have been sampled above. Run the shell script run_sample_cxalbedo.sh. This script calls the code sample_cxalbedo.py, which requires the name of the giant spreadsheet as input.  The names of the giant spreadsheet files are hardcoded in run_sample_cxalbedo.sh.  

sample_cxalbedo.py relies on the GIANT class to calculate the CX BRDF from the model sampled 10M wind speeds.  The function writes the values to an npz file defined in sample_cxalbedo.py that's based on the giant spreadsheet filename.

## Training the NNR

Once the giant spreadsheed and sampled MERRA-2 and other datasets are prepared, you can now train the neural network.
This is done seperately for each MODIS retrieval algorithm with the following python programs:
train_land.py
train_deep.py
train_ocean.py

Within these scripts there are options for inputs, targets, and how to set up the neural network. These are explained in the comments of the python programs. They require that the MERRA-2 and other sampled npz files are located in the same directory as the giant file.

Here are some things to note:
1. the scripts can test different combinations of inputs when you set the 'combinations' variable to True. The Input_const variable list is used in the all the tested combinations. The code takes the Input_nnr list of variables and creates all possible combitions with the Input_const list.  If 'combinations' is set to False, it trains on the combined full list of Input_nnr and Input_const
2. the script can implement training and testing in k-folds by setting the variables K to an integer. Otherwise, if set to none it will train on the balanced subset and test on the entire dataset. Balancing is done by making sure that no aerosol type makes up more than 35% of the total number of obs.
3. The code intializes and 'ABC' class defined in the abc_c6.py program. The ABC classes have a default list of variables that it filters the observations by. These are hardcoded in the abc_c6.py program, and varies by the retrieval. This is the list of variables that it checks for valid values automatically


- LAND: 
        qa == 3 
        AERONET obs aTau470, 550, 660 > -0.01
        MODIS retrieved mTau470, 550, 660, 2100 > -0.01
        MODIS cloud fraction >=0 and < 0.70
        MODIS scattering angle < 170.
        MODIS obs reflectance mRef412, 440, 470, 550, 660, 870, 1200, 1600, 2100 > 0
        MODIS reported surface reflectance mSre470, 660, 2100 > 0    
- DEEP: 
        qa == 3
        AERONET obs aTau470, 550, 660 > -0.01
        MODIS retrieved mTau412, 470, 550, 660 > -0.01
        MODIS cloud fraction >=0 and < 0.70
        MODIS scattering angle < 170.
        MODIS obs reflectance mRef412, 470, 660 > 0
        MODIS reported surface reflectance mSre412, 470, 660 > 0
- OCEAN:
        qa > 0
        AERONET obs aTau470, 550, 660, 870 > -0.01
        MODIS retrieved mTau470, 550, 660, 870 > -0.01
        MODIS cloud fraction >=0 and < 0.70
        MODIS glint angle > 40
        MODIS obs reflectance mRef470, 550, 660, 870, 1200, 1600, 2100 > 0
        

You can add additional variables to filter by with the 'aFilter' variable in train_X.py. Otherwise, 'aFilter' can be just set to None.

The ABC classes also have an optional 'tymemax' variable that if set cuts off the MODIS-AERONET colocation data record at the date given.

3. The next thing that the ABC classes do is remove outliers.  It's based on log-transformed AOD at 550 nm. First it calculates the difference between the MODIS retrieved and AERONET log(AOD550+0.01). It filters any observations where the difference is more than 3 standard deviations from the mean difference. The code does 3 iterations of this - recalculating the mean and standard deviation each time with the reduced dataset.
4. Next step is to convert all observations angles to cos(angle)

After the QA checks and data transformations, the code trains the neural network, and if testing is asked for in the train_X.py script, it will output diagnostic plots.